Домашнее задание по теме «Рекомендации на основе содержания» Использовать dataset MovieLens

Построить рекомендации (регрессия, предсказываем оценку) на фичах:
- TF-IDF на тегах и жанрах
- Средние оценки (+ median, variance, etc.) пользователя и фильма
- Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.info()
links

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [4]:
movies.info()
movies

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
ratings.info()
ratings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
tags.info()
tags

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


## Жанры

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(movies['genres'])
print(vectorizer.get_feature_names())
print(X.shape)

['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi', 'film', 'genres', 'horror', 'imax', 'listed', 'musical', 'mystery', 'no', 'noir', 'romance', 'sci', 'thriller', 'war', 'western']
(9742, 24)


In [8]:
tfidf_genres = pd.DataFrame.sparse.from_spmatrix(
    X,
    index=movies['title'],
    columns=vectorizer.get_feature_names()
)

tfidf_genres

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Jumanji (1995),0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Grumpier Old Men (1995),0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
Waiting to Exhale (1995),0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
Father of the Bride Part II (1995),0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic (2017),0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
No Game No Life: Zero (2017),0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Flint (2017),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [9]:
def print_top_movies_by_genre(genre, count=10):
    movies = list(tfidf_genres[genre].sort_values(ascending=False).head(count).index)
    for i, movie in enumerate(movies):
        print(f'{i + 1}.\t{movie}')

In [10]:
print_top_movies_by_genre('animation', 25)

1.	DC Super Hero Girls: Hero of the Year (2016)
2.	Investigation Held by Kolobki (1986)
3.	Hulk Vs. (2009)
4.	Hedgehog in the Fog (1975)
5.	Illusionist, The (L'illusionniste) (2010)
6.	A Plasticine Crow (1981)
7.	The Monkey King (1964)
8.	Garfield's Pet Force (2009)
9.	Genius Party (2007)
10.	Piper (2016)
11.	Asterix: The Land of the Gods (Astérix: Le domaine des dieux) (2014)
12.	The Red Turtle (2016)
13.	Town Called Panic, A (Panique au village) (2009)
14.	Invincible Iron Man, The (2007)
15.	Fireworks, Should We See It from the Side or the Bottom? (2017)
16.	Afro Samurai: Resurrection (2009)
17.	Vacations in Prostokvashino (1980)
18.	LEGO DC Super Hero Girls: Brain Drain (2017)
19.	Merry Madagascar (2009)
20.	Winnie the Pooh and the Day of Concern (1972)
21.	Wolf and Calf (1984)
22.	Winnie the Pooh Goes Visiting (1971)
23.	Winter in Prostokvashino (1984)
24.	Love Live! The School Idol Movie (2015)
25.	Travels of an Ant (1983)


## Теги

In [11]:
tags_grouped = (
    tags
    .groupby('movieId')['tag']
    .apply(lambda x: '|'.join(x))
    .reset_index()
    .merge(movies, how='right', on='movieId')
    [['movieId', 'title', 'tag']]
)
tags_grouped['tag'] = tags_grouped['tag'].fillna('')

tags_grouped

,movieId,title,tag
0,1,Toy Story (1995),pixar|pixar|fun
1,2,Jumanji (1995),fantasy|magic board game|Robin Williams|game
2,3,Grumpier Old Men (1995),moldy|old
3,5,Father of the Bride Part II (1995),pregnancy|remake
4,7,Sabrina (1995),remake
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),
9738,193583,No Game No Life: Zero (2017),
9739,193585,Flint (2017),
9740,193587,Bungo Stray Dogs: Dead Apple (2018),


In [12]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tags_grouped['tag'])
print(vectorizer.get_feature_names())
print(X.shape)

['06', '1900s', '1920s', '1950s', '1960s', '1970s', '1980s', '1990s', '2001', '250', '2d', '70mm', '80', 'aardman', 'abortion', 'about', 'absorbing', 'abstract', 'abuse', 'academy', 'accident', 'achronological', 'acting', 'action', 'actors', 'actress', 'adam', 'adams', 'adaptation', 'addiction', 'adolescence', 'adoption', 'adorable', 'adrien', 'ads', 'adult', 'adultery', 'adventure', 'affleck', 'afghanistan', 'africa', 'agatha', 'age', 'aggressive', 'aging', 'aids', 'al', 'alan', 'alba', 'alcatraz', 'alcoholism', 'alcott', 'alfred', 'alicia', 'aliens', 'allegorical', 'alone', 'alter', 'alternate', 'am', 'amazing', 'america', 'american', 'americans', 'amish', 'amnesia', 'amtrak', 'amy', 'ancient', 'and', 'andrew', 'android', 'androids', 'andy', 'angeles', 'angelina', 'anger', 'animal', 'animated', 'animation', 'anime', 'anne', 'annoying', 'anthology', 'anthony', 'anti', 'apes', 'apocalypse', 'apocalyptic', 'appealing', 'aquarium', 'archaeology', 'ark', 'arnold', 'art', 'arthouse', 'arth

In [13]:
tfidf_tags = pd.DataFrame.sparse.from_spmatrix(
    X,
    index=tags_grouped['title'],
    columns=vectorizer.get_feature_names()
)

tfidf_tags.shape

(9742, 1744)

In [14]:
def print_top_movies_by_tag(tag, count=10):
    movies = list(tfidf_tags[tag].sort_values(ascending=False).head(count).index)
    for i, movie in enumerate(movies):
        print(f'{i + 1}.\t{movie}')

In [15]:
print_top_movies_by_tag('russia')

1.	Ninotchka (1939)
2.	Andrew Dice Clay: Dice Rules (1991)
3.	Matewan (1987)
4.	River Runs Through It, A (1992)
5.	Shampoo (1975)
6.	My Man Godfrey (1957)
7.	Maurice (1987)
8.	McCabe & Mrs. Miller (1971)
9.	Kagemusha (1980)
10.	Bicycle Thieves (a.k.a. The Bicycle Thief) (a.k.a. The Bicycle Thieves) (Ladri di biciclette) (1948)


### Средние оценки по пользователям

In [16]:
user_stats = (
    ratings[['userId', 'rating']]
    .groupby('userId')
    .agg(['size', 'mean'])['rating']
    .rename(columns={'size': 'rating_count', 'mean': 'rating_avg'})
)

user_stats.sort_values(by='rating_count')

,rating_count,rating_avg
userId,,
442,20,1.275000
406,20,3.250000
147,20,3.375000
194,20,3.475000
569,20,4.000000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


## Регрессия

### Средние оценки по фильмам

In [17]:
movie_stats = (
    ratings[['movieId', 'rating']]
    .groupby('movieId')
    .agg(['size', 'mean'])['rating']
    .rename(columns={'size': 'rating_count', 'mean': 'rating_avg'})
)

movie_stats

,rating_count,rating_avg
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429
...,...,...
193581,1,4.000000
193583,1,3.500000
193585,1,3.500000


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(movies['genres'])
print(vectorizer.get_feature_names())
print(X.shape)

['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi', 'film', 'genres', 'horror', 'imax', 'listed', 'musical', 'mystery', 'no', 'noir', 'romance', 'sci', 'thriller', 'war', 'western']
(9742, 24)


In [19]:
tfidf_genres = pd.DataFrame.sparse.from_spmatrix(
    X,
    index=movies['movieId'],
    columns=vectorizer.get_feature_names()
)

tfidf_genres

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
193583,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
193585,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [20]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tags_grouped['tag'])
print(vectorizer.get_feature_names())
print(X.shape)

['06', '1900s', '1920s', '1950s', '1960s', '1970s', '1980s', '1990s', '2001', '250', '2d', '70mm', '80', 'aardman', 'abortion', 'about', 'absorbing', 'abstract', 'abuse', 'academy', 'accident', 'achronological', 'acting', 'action', 'actors', 'actress', 'adam', 'adams', 'adaptation', 'addiction', 'adolescence', 'adoption', 'adorable', 'adrien', 'ads', 'adult', 'adultery', 'adventure', 'affleck', 'afghanistan', 'africa', 'agatha', 'age', 'aggressive', 'aging', 'aids', 'al', 'alan', 'alba', 'alcatraz', 'alcoholism', 'alcott', 'alfred', 'alicia', 'aliens', 'allegorical', 'alone', 'alter', 'alternate', 'am', 'amazing', 'america', 'american', 'americans', 'amish', 'amnesia', 'amtrak', 'amy', 'ancient', 'and', 'andrew', 'android', 'androids', 'andy', 'angeles', 'angelina', 'anger', 'animal', 'animated', 'animation', 'anime', 'anne', 'annoying', 'anthology', 'anthony', 'anti', 'apes', 'apocalypse', 'apocalyptic', 'appealing', 'aquarium', 'archaeology', 'ark', 'arnold', 'art', 'arthouse', 'arth

In [21]:
tfidf_tags = pd.DataFrame.sparse.from_spmatrix(
    X,
    index=tags_grouped['movieId'],
    columns=vectorizer.get_feature_names()
)

tfidf_tags

,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,250,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
movies_ext = (
    movies[['movieId', 'title']]
    .set_index('movieId')
    .join(tfidf_genres)
    .join(tfidf_tags, lsuffix='_genre', rsuffix='_tag')
    .join(movie_stats)
)
movies_ext['rating_count'] = movies_ext['rating_count'].fillna(0)
movies_ext['rating_avg'] = movies_ext['rating_avg'].fillna(0)

movies_ext

,title,action_genre,adventure_genre,animation_genre,children_genre,comedy_genre,crime_genre,documentary_genre,drama_genre,fantasy_genre,...,younger,your,zellweger,zither,zoe,zombie,zombies,zooey,rating_count,rating_avg
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.0,3.920930
2,Jumanji (1995),0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,3.431818
3,Grumpier Old Men (1995),0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,3.259615
4,Waiting to Exhale (1995),0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.357143
5,Father of the Bride Part II (1995),0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,3.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.000000
193583,No Game No Life: Zero (2017),0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.500000
193585,Flint (2017),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.500000


In [23]:
from sklearn.model_selection import train_test_split

X = movies_ext.copy().drop(['title', 'rating_avg'], axis=1)
y = movies_ext['rating_avg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(9742, 1769) (9742,)
(6527, 1769) (6527,)
(3215, 1769) (3215,)


In [24]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
reg.score(X_test, y_test)

0.06946780923657458

In [25]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred, squared=False)

0.8532462053411107

In [26]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.06946780923657458